# <u>Assignment 1</u>


In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.compose import make_column_selector, make_column_transformer

import warnings
warnings.filterwarnings('ignore')

In [391]:
df = pd.read_csv(r'..\Data Sets\Loan_approval\train.csv')
df.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [392]:
df.set_index('id', inplace=True)

In [393]:
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
id,,,,,,,,,,,,
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [394]:
df_test = pd.read_csv(r'..\Data Sets\Loan_approval\test.csv')
df_test.set_index('id', inplace=True)


In [395]:
X = df.drop(columns=['loan_status'])
y = df['loan_status'].astype('category')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24, stratify=y)

In [396]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46916 entries, 58018 to 40648
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  46916 non-null  int64  
 1   person_income               46916 non-null  int64  
 2   person_home_ownership       46916 non-null  object 
 3   person_emp_length           46916 non-null  float64
 4   loan_intent                 46916 non-null  object 
 5   loan_grade                  46916 non-null  object 
 6   loan_amnt                   46916 non-null  int64  
 7   loan_int_rate               46916 non-null  float64
 8   loan_percent_income         46916 non-null  float64
 9   cb_person_default_on_file   46916 non-null  object 
 10  cb_person_cred_hist_length  46916 non-null  int64  
dtypes: float64(3), int64(4), object(4)
memory usage: 4.3+ MB


In [397]:
X_train.isnull().sum()

person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64

In [398]:
X_train.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
id,,,,,,,,,,,
58018,27,55000,MORTGAGE,11.0,MEDICAL,B,5000,11.86,0.09,N,7
13494,23,91000,MORTGAGE,7.0,DEBTCONSOLIDATION,B,15000,10.74,0.16,N,3
48953,35,90000,RENT,4.0,DEBTCONSOLIDATION,A,20000,6.54,0.22,N,10
12149,24,35000,MORTGAGE,8.0,EDUCATION,A,1500,7.51,0.04,N,4
2376,27,50000,MORTGAGE,6.0,MEDICAL,B,15000,11.26,0.30,N,6


In [399]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
 
ct_ohe = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=[
        'object',
        'category',
    ])),
    (ohe, make_column_selector(dtype_include=[
        'object',
        'category',
    ])),
    verbose_feature_names_out=False,
).set_output(transform='pandas')


## Logistic Regression

In [403]:
logr = LogisticRegression()

pipe_logr = Pipeline([('ct', ct_ohe), ('lr', logr)])

params = {
    "lr__solver": [
        'lbfgs',
        'liblinear',
        'newton-cg',
        'newton-cholesky',
        'sag',
        'saga',
   ],
    "lr__multi_class": [
        "ovr",
        "multinomial"
    ],
    "lr__C" : np.linspace(0.001, 10, 10)
}

kfold = StratifiedKFold(n_splits=5, random_state=24, shuffle=True)

In [404]:
gcv_logr = GridSearchCV(
    estimator=pipe_logr,
    param_grid=params,
    cv=kfold,
)


In [405]:
# Now perform model training and evaluation
gcv_logr.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('ct',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000016BA4A87EC0>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transfor...selector object at 0x0000016BA4BE9A30>)],
                                                          verbose_feature_names_out=False)),
                                       ('lr', LogisticRegression())]),
             param_grid={'lr__C': array([1.000e-03, 1.112e+00, 2.223e+00, 3.334e+00, 4.445e+00, 5.556e+00,
       6.667e+00, 7.778e+00, 8.889e+00, 1.000e+01]),
                         'lr__multi_class': ['ovr', 'multinomial'],
                         'lr__solver': ['lbfgs', 'liblinear', 'newton-cg',
                                        'newton-cholesky', 'sag', 'saga']})

In [444]:
print("Logistic Reg. best params: ", gcv_logr.best_params_)
print("Logistic Reg. best score: ", gcv_logr.best_score_)

Logistic Reg. best params:  {'lr__C': 8.889, 'lr__multi_class': 'ovr', 'lr__solver': 'newton-cholesky'}
Logistic Reg. best score:  0.9119703275200047


In [443]:
best_logr = gcv_logr.best_estimator_
y_pred_proba = best_logr.predict_proba(X_test)[:, 1]
print("Logistic Reg. roc_auc_score:", roc_auc_score(y_test, y_pred_proba))

Logistic Reg. roc_auc_score: 0.8969811644233157


## KNN

In [416]:
scaler_mm = MinMaxScaler()
scaler_std = StandardScaler()

knn = KNeighborsClassifier()

pipe_knn = Pipeline(
    [
        ('CT', ct_ohe),
        ('SCL', None),
        ('KNN', knn)
    ]
)

params = {
    'SCL':[scaler_mm, scaler_std, None],
    'KNN__n_neighbors': np.arange(1, 11, 1)
}

scoring_metrics = {
    'neg_log_loss': 'neg_log_loss',
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1'
}

In [417]:

gcv_knn = GridSearchCV(
    pipe_knn, 
    param_grid=params,
    scoring=scoring_metrics,
    cv=kfold,
    refit='neg_log_loss'  # Specify which metric to use for selecting the best model
)


In [418]:
gcv_knn.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000016BA4A87EC0>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transfor...ject at 0x0000016BA4BE9A30>)],
                                                          verbose_feature_names_out=False)),
                                       ('SCL', None),
                                       ('KNN', KNeighborsClassifier())]),
             param_grid={'KNN__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'SCL': [MinMaxScaler(), StandardScaler(), None]},
             refit='neg_log_loss',
             scoring={'accuracy': 'accuracy', 'f1': 'f1',
                      'neg_log_loss': 'neg_log_loss', 'precision': 'precision',
                      'recall': 'recall'})

In [441]:
print("KNN best_params: ", gcv_knn.best_params_)
print("KNN best_score", gcv_knn.best_score_)


KNN best_params:  {'KNN__n_neighbors': 10, 'SCL': StandardScaler()}
KNN best_score -0.7577327468105872


In [442]:
best_knn = gcv_knn.best_estimator_

y_pred_proba = best_knn.predict_proba(X_test)[:, 1]
print("KNN Reg. roc_auc_score:", roc_auc_score(y_test, y_pred_proba))

KNN Reg. roc_auc_score: 0.8877489280312028


## Gaussian NB

In [427]:
kfold = StratifiedKFold(n_splits=5, random_state=24, shuffle=True)

# Naive Baye: Gaussian
nb = GaussianNB()

pipe_nb = Pipeline([
    ('CT', ct_ohe), 
    ('NB', nb)
])

gcv_nb = GridSearchCV(
    estimator=pipe_nb, 
    cv=kfold,
    scoring='roc_auc',
    param_grid={ 
        'NB__var_smoothing': np.linspace(0.001, 0.1, 10),
    },
)


In [445]:
gcv_nb.fit(X_train, y_train)
print('GaussianNB best score: ',gcv_nb.best_params_)
print('GaussianNB best score: ',gcv_nb.best_score_)

GaussianNB best score:  {'NB__var_smoothing': 0.012}
GaussianNB best score:  0.7391018519253109


In [446]:
best_nb = gcv_nb.best_estimator_

y_pred_proba = best_nb.predict_proba(X_test)[:, 1]
print("GaussianNB roc_auc_score:", roc_auc_score(y_test, y_pred_proba))

GaussianNB roc_auc_score: 0.7207805682997263
